In [56]:
# Import Packages
import numpy as np
from ete3 import Tree, TreeStyle, NodeStyle, TextFace

In [46]:
class Node:
    def __init__(self, name, length=0.0):
        self.name = name      # Node name
        self.length = length  # branch length = from parent to self
        self.left = None      # left child Node
        self.right = None     # right child Node

def add_child(parent, direction="left", edge_length=0.0):
    # Generate name
    child_name = parent.name + ("0" if direction=="left" else "1")
    
    child = Node(child_name, length=edge_length)
    
    # Connect to parent
    if direction == "left":
        parent.left = child
    else:
        parent.right = child
    
    return child

def count_leaves(node):
    if node is None:
        return 0
    if node.left is None and node.right is None:
        return 1
    return count_leaves(node.left) + count_leaves(node.right)

def to_newick(node):
    if node.left is None and node.right is None:
        # leaf node
        return f"{node.name}:{node.length}"
    
    parts = []
    if node.left:
        parts.append(to_newick(node.left))
    if node.right:
        parts.append(to_newick(node.right))
    
    # internal node: "(left,right)name:length"
    name_part = node.name if node.name else ""
    return f"({','.join(parts)}){name_part}:{node.length}"

생성 방법론
1. Aldous Beta-splitting
2. Birth-death model
3. Yule model = Pure Birth model
4. Kingman Coalescent model
5. Uniform/Random binary tree model

Single Tree Statistic
1. number of leaves
2. number of internal nodes
3. Tree height
4. subtree sizes
5. Colless index
6. Sackins index

Tree to tree comparison
1. Robinson-Foulds (RF) distance
2. Weighted, normalized RF distance
3. Branch score distance
4. K-tree score
5. Quartet/triplet distance

In [58]:
## SAMPLE TREE GENERATION
root = Node("", length=0.0)

# 첫 분기: left 3 leaf, right 5 leaf
left_subtree = add_child(root, "left", 3)
right_subtree = add_child(root, "right", 5)

# 왼쪽 subtree에 leaf 3개
l1 = add_child(left_subtree, "left", 1)   # "00"
l2 = add_child(left_subtree, "right", 1)  # "01"
add_child(l1, "left", 0.5)                # "000"
add_child(l1, "right", 0.5)               # "001"
add_child(l2, "left", 0.5)                # "010"

# 오른쪽 subtree에 leaf 5개
r1 = add_child(right_subtree, "left", 2)  # "10"
r2 = add_child(right_subtree, "right", 2) # "11"
add_child(r1, "left", 0.5)                # "100"
add_child(r1, "right", 0.5)               # "101"
add_child(r2, "left", 0.5)                # "110"
r2_right = add_child(r2, "right", 0.5)    # "111"
add_child(r2_right, "left", 0.2)          # "1110"
add_child(r2_right, "right", 0.2)

newick_str = to_newick(root) + ";"
# print(newick_str)

t = Tree(newick_str)

ts = TreeStyle()
ts.show_branch_length = True  # branch length 표시
ts.show_leaf_name = True      # leaf 이름 표시

# internal node 이름 표시
for n in t.traverse():
    ns = NodeStyle()
    ns["size"] = 0  # 원 숨김
    n.set_style(ns)
    if not n.is_leaf():
        # TextFace를 branch 오른쪽에 붙임
        n.add_face(TextFace(n.name, fsize=10, fgcolor="red"), column=0, position="branch-right")

t.show(tree_style=ts)

# print(t.get_ascii())

# print("Root leaf_count:", count_leaves(root))

In [62]:
num_leaves = 100  # 원하는 leaf 개수
width = len(str(num_leaves))
leaf_names = [f"{i:0{width}}" for i in range(1, num_leaves + 1)]